In [1]:
"""%matplotlib inline
import os; import sys
import math; import copy
import joblib; import glob
import numpy as np; from numpy import load, array, argmax
import pandas as pd
import scanpy as sc
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt


import scipy as sp
from scipy import ndimage as ndi


from pathlib import Path
import stlearn as st
st.settings.set_figure_params(dpi=300)
from stimage._utils import gene_plot, tiling


from pathlib import Path
from typing import Optional, Union
from anndata import AnnData


import lime
from lime import lime_image
from PIL import Image
from skimage.color import rgb2hed
from skimage.segmentation import mark_boundaries
from skimage.segmentation import watershed
import skimage
from skimage.color import rgb2hed
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage.measure import label
from skimage.morphology import area_opening


from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.cluster import AgglomerativeClustering
from sklearn.neural_network import MLPClassifier, MLPRegressor


import tensorflow as tf
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image as image_fun


from imblearn.over_sampling import SMOTE
from tabgan.sampler import OriginalGenerator, GANGenerator
import lightgbm as lgb"""

'%matplotlib inline\nimport os; import sys\nimport math; import copy\nimport joblib; import glob\nimport numpy as np; from numpy import load, array, argmax\nimport pandas as pd\nimport scanpy as sc\nimport seaborn as sns\nfrom tqdm import tqdm\nimport matplotlib.pyplot as plt\n\n\nimport scipy as sp\nfrom scipy import ndimage as ndi\n\n\nfrom pathlib import Path\nimport stlearn as st\nst.settings.set_figure_params(dpi=300)\nfrom stimage._utils import gene_plot, tiling\n\n\nfrom pathlib import Path\nfrom typing import Optional, Union\nfrom anndata import AnnData\n\n\nimport lime\nfrom lime import lime_image\nfrom PIL import Image\nfrom skimage.color import rgb2hed\nfrom skimage.segmentation import mark_boundaries\nfrom skimage.segmentation import watershed\nimport skimage\nfrom skimage.color import rgb2hed\nfrom skimage.feature import peak_local_max\nfrom skimage.segmentation import watershed\nfrom skimage.measure import label\nfrom skimage.morphology import area_opening\n\n\nfrom sklea

In [2]:
"""BASE_PATH = Path("D:/Onkar_D/UQ/Project_Spt.Transcriptomics/Output_files")

TILE_PATH = BASE_PATH / "tiles"
TILE_PATH.mkdir(parents=True, exist_ok=True)

TILE_PATH_2 = BASE_PATH / "big_tiles"
TILE_PATH_2.mkdir(parents=True, exist_ok=True)


SAMPLE = "block1"
Sample1 = st.Read10X(BASE_PATH / SAMPLE, 
                  library_id=SAMPLE, 
                  count_file="V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5",
                  quality="fulres",)
img = plt.imread(BASE_PATH / SAMPLE /"V1_Breast_Cancer_Block_A_Section_1_image.tif", 0)
Sample1.uns["spatial"][SAMPLE]['images']["fulres"] = img


SAMPLE = "block2"
Sample2 = st.Read10X(BASE_PATH / SAMPLE, 
                  library_id=SAMPLE, 
                  count_file="V1_Breast_Cancer_Block_A_Section_2_filtered_feature_bc_matrix.h5",
                  quality="fulres",)
                  #source_image_path=BASE_PATH / SAMPLE /"V1_Breast_Cancer_Block_A_Section_1_image.tif")
img = plt.imread(BASE_PATH / SAMPLE /"V1_Breast_Cancer_Block_A_Section_2_image.tif", 0)
Sample2.uns["spatial"][SAMPLE]['images']["fulres"] = img


SAMPLE = "FFPE_Breast_Cancer"
Sample3 = st.Read10X(BASE_PATH / SAMPLE, 
                  library_id=SAMPLE, 
                  count_file="Visium_FFPE_Human_Breast_Cancer_filtered_feature_bc_matrix.h5",
                  quality="fulres",)
img = plt.imread(BASE_PATH / SAMPLE /"Visium_FFPE_Human_Breast_Cancer_image.tif", 0)
Sample3.uns["spatial"][SAMPLE]['images']["fulres"] = img


for adata in [Sample1, Sample2, Sample3]:
    st.pp.filter_genes(adata,min_cells=3)
    st.pp.log1p(adata)
    sc.pp.filter_cells(adata,min_counts=100)
    
    TILE_PATH_ = TILE_PATH / list(adata.uns["spatial"].keys())[0]
    TILE_PATH_.mkdir(parents=True, exist_ok=True)
    tiling(adata, TILE_PATH_, crop_size=299)
    features = st.pp.extract_feature(adata)
    adata.obsm['ResNet50_features_small'] = features
    
for adata in [Sample1, Sample2]:
    st.pp.filter_genes(adata,min_cells=3)
    st.pp.log1p(adata)
    sc.pp.filter_cells(adata,min_counts=100)
    
    TILE_PATH_2_ = TILE_PATH_2 / list(adata.uns["spatial"].keys())[0]
    TILE_PATH_2_.mkdir(parents=True, exist_ok=True)
    tiling(adata, TILE_PATH_2_, crop_size=1950)
    st.pp.extract_feature(adata)
    features = st.pp.extract_feature(adata)
    adata.obsm['ResNet50_features_big'] = features
    
for adata in [Sample3]:
    st.pp.filter_genes(adata,min_cells=3)
    st.pp.log1p(adata)
    sc.pp.filter_cells(adata,min_counts=100)
    
    TILE_PATH_2_ = TILE_PATH_2 / list(adata.uns["spatial"].keys())[0]
    TILE_PATH_2_.mkdir(parents=True, exist_ok=True)
    tiling(adata, TILE_PATH_2_, crop_size=1650)
    st.pp.extract_feature(adata)
    adata.obsm['ResNet50_features_big'] = features
    
    
# Concatenate Features
def concatenate_features(Sample1):
    return pd.concat([[Sample1.obsm['ResNet50_features_small']],[Sample1.obsm['ResNet50_features_big']]], axis=1)

X_train = concatenate_features(Sample1)
X_test = concatenate_features(Sample2)
X_test_ffpe = concatenate_features(Sample3)



# Get the Clustering of all Spots
def Spot_Cluster(Features,n):
    hc1 = AgglomerativeClustering(n_clusters = n, affinity = 'euclidean', linkage = 'ward')
    return hc1.fit_predict(Features)

Spots1_hc = hc1.fit_predict(X_train,5)
Spots2_hc = hc2.fit_predict(X_test,3)
Spots3_hc = hc3.fit_predict(X_test_ffpe,3)"""

'BASE_PATH = Path("D:/Onkar_D/UQ/Project_Spt.Transcriptomics/Output_files")\n\nTILE_PATH = BASE_PATH / "tiles"\nTILE_PATH.mkdir(parents=True, exist_ok=True)\n\nTILE_PATH_2 = BASE_PATH / "big_tiles"\nTILE_PATH_2.mkdir(parents=True, exist_ok=True)\n\n\nSAMPLE = "block1"\nSample1 = st.Read10X(BASE_PATH / SAMPLE, \n                  library_id=SAMPLE, \n                  count_file="V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5",\n                  quality="fulres",)\nimg = plt.imread(BASE_PATH / SAMPLE /"V1_Breast_Cancer_Block_A_Section_1_image.tif", 0)\nSample1.uns["spatial"][SAMPLE][\'images\']["fulres"] = img\n\n\nSAMPLE = "block2"\nSample2 = st.Read10X(BASE_PATH / SAMPLE, \n                  library_id=SAMPLE, \n                  count_file="V1_Breast_Cancer_Block_A_Section_2_filtered_feature_bc_matrix.h5",\n                  quality="fulres",)\n                  #source_image_path=BASE_PATH / SAMPLE /"V1_Breast_Cancer_Block_A_Section_1_image.tif")\nimg = plt.imread

In [3]:
"""def Classifier(Sample1, Sample2, biomarker_list, model_c):
    
    MinMax_scaler_y_1 = preprocessing.MinMaxScaler(feature_range =(0, 1))
    MinMax_scaler_y_2 = preprocessing.MinMaxScaler(feature_range =(0, 1))

    Y_1 = Sample1.to_df()[biomarker_list]
    Y_1 = MinMax_scaler_y_1.fit_transform(Y_1) 
    Y_1 = pd.DataFrame(data=Y_1)
    Y_1 = Y_1.apply(lambda x: pd.cut(x, bins=[-0.01, 0.5, 1.01], labels = [0, 1]))
    Y_1 = Y_1.astype('int64')


    test_Y_1 = Sample2.to_df()[biomarker_list]
    test_Y_1 = MinMax_scaler_y_2.fit_transform(test_Y_1)
    test_Y_1 = pd.DataFrame(data=test_Y_1)
    test_Y_1 = test_Y_1.apply(lambda x: pd.cut(x, bins=[-0.01, 0.5, 1.01], labels = [0, 1]))
    test_Y_1 = test_Y_1.fillna(0)
    test_Y_1 = test_Y_1.astype('int64')

    # Classifier
    clf = MultiOutputClassifier(model_c).fit(X_train, Y_1)
    return clf.predict(X_test)
    #joblib.dump(clf, wd+'New_model_clf_biomarker_10X.pkl')
    
    
biomarker_list_1 = ['COX6C','TTLL12','PGM5','KRT5','SLITRK6', 'CPB1', "PABPC1", "GNAS", "HSP90AB1", "TFF3", "ATP1A1", "B2M", "FASN", "ESR1", "SPARC", "ACTG1", "HLA-B", "LINC00645", "MALAT1"]
biomarker_list_2 = ['COX6C','TTLL12', "PABPC1", "GNAS", "HSP90AB1", "TFF3", "ATP1A1", "B2M", "FASN", "SPARC"]
model_c = MLPClassifier(max_iter=10000)
#model_r = MLPRegressor(max_iter=10000)"""

'def Classifier(Sample1, Sample2, biomarker_list, model_c):\n    \n    MinMax_scaler_y_1 = preprocessing.MinMaxScaler(feature_range =(0, 1))\n    MinMax_scaler_y_2 = preprocessing.MinMaxScaler(feature_range =(0, 1))\n\n    Y_1 = Sample1.to_df()[biomarker_list]\n    Y_1 = MinMax_scaler_y_1.fit_transform(Y_1) \n    Y_1 = pd.DataFrame(data=Y_1)\n    Y_1 = Y_1.apply(lambda x: pd.cut(x, bins=[-0.01, 0.5, 1.01], labels = [0, 1]))\n    Y_1 = Y_1.astype(\'int64\')\n\n\n    test_Y_1 = Sample2.to_df()[biomarker_list]\n    test_Y_1 = MinMax_scaler_y_2.fit_transform(test_Y_1)\n    test_Y_1 = pd.DataFrame(data=test_Y_1)\n    test_Y_1 = test_Y_1.apply(lambda x: pd.cut(x, bins=[-0.01, 0.5, 1.01], labels = [0, 1]))\n    test_Y_1 = test_Y_1.fillna(0)\n    test_Y_1 = test_Y_1.astype(\'int64\')\n\n    # Classifier\n    clf = MultiOutputClassifier(model_c).fit(X_train, Y_1)\n    return clf.predict(X_test)\n    #joblib.dump(clf, wd+\'New_model_clf_biomarker_10X.pkl\')\n    \n    \nbiomarker_list_1 = [\'COX

In [4]:
"""# Spatial Autocorrelation
import geopandas as gpd
import pysal
from pysal.explore import esda
import pysal.lib as lps
from esda.moran import Moran, Moran_Local, Moran_BV, Moran_Local_BV
import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster, plot_moran_bv_simulation, plot_moran_bv, plot_local_autocorrelation
from libpysal.weights.contiguity import Queen
from libpysal import examples"""

'# Spatial Autocorrelation\nimport geopandas as gpd\nimport pysal\nfrom pysal.explore import esda\nimport pysal.lib as lps\nfrom esda.moran import Moran, Moran_Local, Moran_BV, Moran_Local_BV\nimport splot\nfrom splot.esda import moran_scatterplot, plot_moran, lisa_cluster, plot_moran_bv_simulation, plot_moran_bv, plot_local_autocorrelation\nfrom libpysal.weights.contiguity import Queen\nfrom libpysal import examples'